<h2> Step 4:Tried feature selection using sparse PCA.</h2>

In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier  
from sklearn import preprocessing 
from sklearn.decomposition import SparsePCA
import random 
from sklearn.datasets import load_boston
import pandas as pd
import numpy as np 
from sklearn.preprocessing import StandardScaler 
from sklearn.decomposition import PCA 

random.seed(123)

os.getcwd()
os.chdir('/Users/khadekirti/Desktop/Data Science/Final Dataset')

In [2]:
def fitTransform(dfAll):
    # Get column names first
    names = list(dfAll.columns)
    # List 
    names.remove('Country Code')
    names.remove('Year')
    # Data Frame to be scalarised
    scaler = preprocessing.MinMaxScaler()
    scaled_df = scaler.fit_transform(dfAll[names])
    scaled_df = pd.DataFrame(scaled_df, columns= names)
    # Add index
    extra_df = dfAll[['Country Code','Year']]
    scaled_df = scaled_df.reset_index()
    extra_df = extra_df.reset_index()
    # Merge 
    dfAll = pd.merge(extra_df,scaled_df, how = 'inner', on = ['index'])
    del dfAll['index']
    return dfAll

In [3]:
def dataProcessing(df, year):
    dfAggeregateAll = pd.DataFrame(columns = ['Country Code','Year'])
    k = 1  
    df = df[['Country Code','Indicator Code', year]] 
    listIndicatorCode = list(set(list(np.unique(df["Indicator Code"])))) 
    for j in listIndicatorCode:
        df_sub = df.loc[df['Indicator Code'] == j]
        df_sub['Year'] = year
        df_sub[j] = df_sub[year]
        del df_sub[year]
        del df_sub['Indicator Code']
        dfAggeregateAll = dfAggeregateAll.drop_duplicates() 
        if (k == 1):
            dfAggeregateAll = pd.merge(dfAggeregateAll,df_sub, how = "outer", on = ['Country Code', 'Year'])
            k = 2
        else: 
            dfAggeregateAll = pd.merge(dfAggeregateAll,df_sub, how = "outer", on = ['Country Code', 'Year'])
    dfAggeregateAll = dfAggeregateAll.drop_duplicates()       
    return dfAggeregateAll 

In [4]:
def dataVarianceCheck(data): 
    #data = data.fillna(0)
    # take numerical value
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    # select numerical columns
    numeric = data.select_dtypes(include=numerics)
    # get the variance
    var = numeric.var()
    # make a list of numeric columns
    features = numeric.columns
    # get the bounds 
    boundData = pd.DataFrame(var, columns = ['var'])
    minValue = boundData['var'].describe()['25%']
    maxValue = boundData['var'].describe()['max'] *.8
    variable = []
    for i in range(0,len(var)):
        if ((var[i]>= minValue) & (var[i] < maxValue)):
            variable.append(features[i])
    variable = list(set(variable))        
    variable.extend(['Country Code','Year','GDP']) 
    variable = list(set(variable))
    return variable

In [5]:
def PCAFeatureReduction(df):
    # Fill na as zero
    #df=df.fillna(0)
    finalIndicatormax = [] 
    # List columns with numerical values
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    # select numerical columns
    features = list(df.select_dtypes(include=numerics).columns) 
    features.remove('GDP')
    # get x and y
    x = df.loc[:, features].values
    # replace nan with zero and inf with a large number
    x = np.nan_to_num(x)
    y = df.loc[:,['GDP']].values
    #Fitting the PCA algorithm with our Data
    pca = PCA().fit(x)
    #To get the number of component,we see the number of compenets needed to makde explained variance greater than 0.988
    sum_ = 0
    for i in list(pca.explained_variance_ratio_): 
        sum_ += i
        if (sum_ >= 0.988):
            comxponentsNeeded = list(pca.explained_variance_ratio_).index(i)
            break
    # Intialise Sparse PCA for the componentsNeeded
    componentsNeeded = int(componentsNeeded/2)
    transformer = SparsePCA(n_components= componentsNeeded,normalize_components=True,random_state=0)
    transformer.fit(x) 
    # get the componpca.components, this will have componentsNeeded as rows and feature as colimns 
    dataPCA = pd.DataFrame(data = transformer.components_,columns = features) 
    # reset index
    dataPCA = dataPCA.reset_index()
    # get a list of indicators with max correlation in the 
    for index in range(componentsNeeded):
        maxColrelation = 0
        # get the dataframe
        dataPCAsub = dataPCA[dataPCA['index'] == index]
        # select the columns with greater than zero 
        dataPCAsub = dataPCAsub.loc[:, (dataPCAsub != 0).any(axis=0)]
        # get the columns
        columnList = list(dataPCAsub.columns)
        if ('index' in columnList): 
            columnList.remove('index')
        if columnList != None: 
            # get the max colrealted column     
            for col in columnList: 
                corelation = abs(df[col].corr(df['GDP']))
                if corelation > maxColrelation: 
                    max_ = col 
            # append in the list    
            finalIndicatormax.append(max_)
    finalIndicatormax.extend(['Country Code','Year','GDP'])  
    return finalIndicatormax


In [6]:
def getFeatures(data): 
    b = data.columns.get_loc("GDP")
    # for all columns, apart from GDP, replace negative values with zero
    columns = data.select_dtypes(include=[np.number]).columns.tolist()
    columns.remove('GDP')
    X = data[columns]
    y = data.iloc[ : , b].values
    featureSelection = stepwise_selection(X, y)
    return featureSelection

In [7]:
dfGDPAll = pd.DataFrame()
for year in ['2003', '2004','2005', '2006', '2007', '2008', '2009','2010', '2011', '2012', '2013', '2014','2015','2016']: 
    for i in os.listdir(): 
        if ("GDP" in i):
            dfGDP = pd.read_csv( os.getcwd() + '/GDP.csv' , skiprows=4)
            del dfGDP['Unnamed: 63']
            dfGDP = dfGDP[['Country Code','Indicator Code', year]]
            listIndicatorCode = list(np.unique(dfGDP["Indicator Code"]))
            dfGDP['Year'] = year
            dfGDP['GDP'] = dfGDP[year]
            del dfGDP[year]
            del dfGDP['Indicator Code']
            dfGDPAll = dfGDPAll.append(dfGDP)

In [8]:
dfGDPAll.head()

,Country Code,Year,GDP
0,ABW,2003,2.470954e+09
1,AFG,2003,8.124454e+09
2,AGO,2003,4.391367e+10
3,ALB,2003,8.283248e+09
4,AND,2003,3.331207e+09


In [18]:
dfAllAggregation = pd.DataFrame(columns = ['Country Code', 'Year'])
listAllColumns = ['Country Code', 'Year'] 
m = 1
for i in os.listdir(): 
    if (('csv' in i) & ('final' in i)) :
        print(i)
        df = pd.read_csv( os.getcwd() + '/' + i)
        del df['Unnamed: 0']
        del df['V63']
        del df['V62']
        df.columns  = ['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code', '2003', '2004','2005', '2006', '2007', '2008', '2009','2010', '2011', '2012', '2013', '2014','2015', '2016']
        dfAllYear = pd.DataFrame() 
        for year in ['2003', '2004','2005', '2006', '2007', '2008', '2009','2010', '2011', '2012', '2013', '2014','2015', '2016']: 
            print("Check")
            # List the Indicator Code 
            # Process the dataframe 
            dfAggeregateAll = dataProcessing(df,year)
            #Removing columns with any misisng values 
            dfAggeregateAll = dfAggeregateAll.loc[:, dfAggeregateAll.isnull().mean() == 0]  
            # merge 
            dfAll =  pd.merge(dfGDPAll,dfAggeregateAll, how = 'inner', on = ['Country Code', 'Year'])  
            # Standarise the numerical value
            dfAll = fitTransform(dfAll)
            # append
            dfAllYear = dfAllYear.append(dfAll)
            # drop duplicates 
            dfAllYear = dfAllYear.drop_duplicates()
        print("Check1")
        # check ones again     
        dfAllYear = dfAllYear.drop_duplicates()    
        # For the final append remving huge missing  
        dfAllYear = dfAllYear.loc[:, dfAllYear.isnull().mean() < 0.2]  
        # Replacing others by mean 
        dfAllYear = dfAllYear.fillna(dfAllYear.mean())
        #Remome very high and very low variance 
        varienceBond =  dataVarianceCheck(dfAllYear)
        dfAllYear = dfAllYear[varienceBond]
        # PCA 
        PCAFeatues = PCAFeatureReduction(dfAllYear)
        dfAllYear = dfAllYear[PCAFeatues]
        print("check2")
        # For final aggregation
        if (m == 1): 
            dfAllAggregation = pd.merge(dfAllAggregation,dfAllYear, how = "outer",on = ['Country Code', 'Year'])
            m = 2
        else:
            # list of indicator already present in dfAllAggregation should be removed
            dfAllYearCol = dfAllYear.columns
            dfAllAggregationCol = dfAllAggregation.columns
            dfColFinal = [x for x in dfAllYearCol if x not in dfAllAggregationCol]
            dfColFinal.extend(['Country Code', 'Year'])
            dfAllYear = dfAllYear[dfColFinal]
            if len(dfColFinal) >  2: 
                print("Adding")
                dfAllAggregation = pd.merge(dfAllAggregation,dfAllYear, how = "outer",on = ['Country Code', 'Year']) 
    else: 
        print("This did not have any feature to be selected: " + i)

This did not have any feature to be selected: error_2 Aid Effectiveness.csv
This did not have any feature to be selected: error_1 Economy & Growth.csv
This did not have any feature to be selected: error_2 Agriculture.csv
final Health.csv
Check


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


Check


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the 

Check


...

Check


/usr/local/lib/python3.7/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-

Check


/usr/local/lib/python3.7/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


Check1
check2
Adding
This did not have any feature to be selected: External Debt.csv


In [19]:
dfAllAggregation = dfAllAggregation.loc[:,~dfAllAggregation.columns.duplicated()]

In [22]:
dfAllAggregation.head()

,SH.PRG.ANEM,SH.STA.MALN.FE.ZS,SH.STA.WAST.MA.ZS,SP.REG.BRTH.UR.ZS,SH.FPL.SATM.ZS,SP.DYN.WFRT,SH.STA.SMSS.UR.ZS,SH.STA.HYGN.UR.ZS,SH.TBS.DTEC.ZS,SH.UHC.NOP1.ZS,...,DT.DOD.MIBR.CD,DT.GPA.PRVT,AG.LND.AGRI.ZS,DT.DOD.DECT.EX.ZS,DT.CUR.SDRW.ZS,DT.GPA.DPPG,DT.INR.OFFT,DT.CUR.OTHC.ZS,DT.MAT.OFFT,DT.DOD.DSTC.XP.ZS
0,0.808061,0.472289,1.000000,0.684485,0.000000,0.945946,1.000000,0.224115,0.192982,0.004399,...,0.0,0.000000,0.124984,0.039541,0.440233,0.027596,0.191991,0.049908,0.540548,0.017395
1,0.533589,0.130120,0.358852,0.000000,0.946860,0.162162,0.434498,0.913738,0.087719,0.031022,...,0.0,0.000000,0.678903,0.041672,0.000000,0.000000,0.000000,0.000000,0.000000,0.088512
2,0.808061,0.604819,0.047847,0.684485,0.919485,0.945946,0.400281,0.300013,0.206140,0.955477,...,0.0,0.017619,0.538837,0.027829,0.175474,0.035943,0.555909,0.028350,0.340664,0.052715
3,0.245681,0.506024,0.210526,0.684485,0.370370,0.162162,0.617444,0.098685,0.328947,0.138992,...,0.0,0.013389,0.476533,0.046859,0.112197,0.515959,0.287080,0.069488,0.540548,0.068452
4,0.159309,0.007229,0.636364,1.000000,0.919485,0.162162,0.281673,0.274425,0.328947,0.004399,...,0.0,0.934850,0.567671,0.029918,0.300609,0.397821,0.476736,0.275476,0.604200,0.176041


In [21]:
dfAllAggregation.shape

(3696, 254)

In [24]:
dfAllAggregation['Year'].drop_duplicates()

0       2003
264     2004
528     2005
792     2006
1056    2007
1320    2008
1584    2009
1848    2010
2112    2011
2376    2012
2640    2013
2904    2014
3168    2015
3432    2016
Name: Year, dtype: object

In [25]:
PCAFeatues = PCAFeatureReduction(dfAllAggregation)
dfAllAggregationFeature = dfAllAggregation[PCAFeatues]

In [26]:
dfAllAggregationFeature.head()

,DT.CUR.OTHC.ZS,per_si_allsi.cov_q5_tot,DT.TDS.DECT.EX.ZS,SE.SEC.TCAQ.LO.ZS,SL.TLF.0714.ZS,DT.DOD.DSTC.XP.ZS,SL.UEM.1524.FE.ZS,DT.CUR.OTHC.ZS,SL.TLF.0714.SW.MA.TM,SL.SLF.0714.MA.ZS,...,SE.SEC.CUAT.UP.ZS,DT.CUR.SDRW.ZS,SE.SEC.TCAQ.UP.ZS,DT.CUR.SDRW.ZS,SE.SEC.ENRL.UP.TC.ZS,SH.STA.SMSS.ZS,SH.STA.OWGH.ZS,Country Code,Year,GDP
0,0.049908,0.0,0.165826,0.502415,0.000000,0.017395,0.419055,0.049908,0.393765,0.336508,...,0.491797,0.440233,0.121108,0.440233,0.150580,0.713193,0.000000,ABW,2003,0.000045
1,0.000000,0.0,0.176338,0.434035,0.292778,0.088512,0.065157,0.000000,0.393765,0.336508,...,0.760218,0.000000,1.000000,0.000000,0.228179,0.596984,0.666667,AFG,2003,0.000151
2,0.028350,1.0,0.216791,0.783768,1.000000,0.052715,0.882361,0.028350,0.393765,0.336508,...,0.491797,0.175474,1.000000,0.175474,0.090108,0.669902,0.357724,AGO,2003,0.000817
3,0.069488,0.0,0.070707,0.184331,0.041394,0.068452,0.382963,0.069488,0.393765,0.336508,...,0.592851,0.112197,0.975702,0.112197,0.313140,0.549401,0.666667,ALB,2003,0.000154
4,0.275476,0.0,0.212195,0.974667,1.000000,0.176041,0.625581,0.275476,0.393765,0.336508,...,0.000000,0.300609,0.562326,0.300609,0.175742,0.295088,0.357724,AND,2003,0.000061


In [27]:
dfAllAggregationFeature.shape

(3696, 113)

In [28]:
dfAllAggregationFeature.to_csv("dfAllAggregationFeature4.csv")

In [29]:
dfAllAggregation.to_csv("dfAllAggregation4.csv")

In [48]:
def PCAFeatureReduction(df):
    # Fill na as zero
    #df=df.fillna(0)
    finalIndicatormax = [] 
    # List columns with numerical values
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    # select numerical columns
    features = list(df.select_dtypes(include=numerics).columns) 
    features.remove('GDP')
    # get x and y
    x = df.loc[:, features].values
    # replace nan with zero and inf with a large number
    x = np.nan_to_num(x)
    y = df.loc[:,['GDP']].values
    #Fitting the PCA algorithm with our Data
    pca = PCA().fit(x)
    #To get the number of component,we see the number of compenets needed to makde explained variance greater than 0.988
    sum_ = 0
    for i in list(pca.explained_variance_ratio_): 
        sum_ += i
        if (sum_ >= 0.988):
            comxponentsNeeded = list(pca.explained_variance_ratio_).index(i)
            break
    # Intialise Sparse PCA for the componentsNeeded
    componentsNeeded = 30
    transformer = SparsePCA(n_components= 30,normalize_components=True,random_state=0)
    transformer.fit(x) 
    # get the componpca.components, this will have componentsNeeded as rows and feature as colimns 
    dataPCA = pd.DataFrame(data = transformer.components_,columns = features) 
    # reset index
    dataPCA = dataPCA.reset_index()
    # get a list of indicators with max correlation in the 
    for index in range(30):
        maxColrelation = 0
        # get the dataframe
        dataPCAsub = dataPCA[dataPCA['index'] == index]
        # select the columns with greater than zero 
        dataPCAsub = dataPCAsub.loc[:, (dataPCAsub != 0).any(axis=0)]
        # get the columns
        columnList = list(dataPCAsub.columns)
        if ('index' in columnList): 
            columnList.remove('index')
        if columnList != None: 
            # get the max colrealted column     
            for col in columnList: 
                corelation = abs(df[col].corr(df['GDP']))
                if corelation > maxColrelation: 
                    max_ = col 
            # append in the list    
            finalIndicatormax.append(max_)
    finalIndicatormax.extend(['Country Code','Year','GDP'])  
    return finalIndicatormax


In [44]:
PCAFeatues = PCAFeatureReduction(dfAllAggregation)
dfAllAggregationFeature50 = dfAllAggregation[PCAFeatues]

In [45]:
dfAllAggregationFeature50.to_csv("dfAllAggregationFeature50.csv")

In [53]:
dfAllAggregationFeature50.shape

(3696, 53)

In [85]:
dfAllAggregationFeature30 = dfAllAggregationFeature30.loc[:,~dfAllAggregationFeature30.columns.duplicated()]
dfAllAggregationFeature30.shape

(3696, 27)

In [83]:
dfAllAggregationFeature50 = dfAllAggregationFeature50.loc[:,~dfAllAggregationFeature50.columns.duplicated()]
dfAllAggregationFeature50.shape

(3696, 43)

In [49]:
PCAFeatues = PCAFeatureReduction(dfAllAggregation)
dfAllAggregationFeature30 = dfAllAggregation[PCAFeatues]

In [50]:
dfAllAggregationFeature30.to_csv("dfAllAggregationFeature30.csv")

In [52]:
dfAllAggregationFeature30.shape

(3696, 33)

In [54]:
dfAllAggregation.shape

(3696, 254)

In [16]:
dfAllAggregation = pd.DataFrame(columns = ['Country Code', 'Year'])
listAllColumns = ['Country Code', 'Year'] 
dfcorrelated = pd.DataFrame()
m = 1
for i in os.listdir(): 
    if (('csv' in i) & ('final' in i)) :
        print(i)
        df = pd.read_csv( os.getcwd() + '/' + i)
        del df['Unnamed: 0']
        del df['V63']
        del df['V62']
        df.columns  = ['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code', '2003', '2004','2005', '2006', '2007', '2008', '2009','2010', '2011', '2012', '2013', '2014','2015', '2016']
        dfAllYear = pd.DataFrame() 
        for year in ['2003', '2004','2005', '2006', '2007', '2008', '2009','2010', '2011', '2012', '2013', '2014','2015', '2016']: 
            print("Check")
            # List the Indicator Code 
            # Process the dataframe 
            dfAggeregateAll = dataProcessing(df,year)
            #Removing columns with any misisng values 
            dfAggeregateAll = dfAggeregateAll.loc[:, dfAggeregateAll.isnull().mean() == 0]  
            # merge 
            dfAll =  pd.merge(dfGDPAll,dfAggeregateAll, how = 'inner', on = ['Country Code', 'Year'])  
            dfAll = fitTransform(dfAll)
            # append
            dfAllYear = dfAllYear.append(dfAll)
            # drop duplicates 
            dfAllYear = dfAllYear.drop_duplicates()
            # data correlated 
            dfcorrelated = dfcorrelated.append(dfAllYear.corr())
        print("Check1")
        # check ones again     
        dfAllYear = dfAllYear.drop_duplicates()    
        print("check2")
        # For final aggregation
        if (m == 1): 
            dfAllAggregation = pd.merge(dfAllAggregation,dfAllYear, how = "outer",on = ['Country Code', 'Year'])
            m = 2
        else:
            # list of indicator already present in dfAllAggregation should be removed
            dfAllYearCol = dfAllYear.columns
            dfAllAggregationCol = dfAllAggregation.columns
            dfColFinal = [x for x in dfAllYearCol if x not in dfAllAggregationCol]
            dfColFinal.extend(['Country Code', 'Year'])
            dfAllYear = dfAllYear[dfColFinal]
            if len(dfColFinal) >  2: 
                print("Adding")
                dfAllAggregation = pd.merge(dfAllAggregation,dfAllYear, how = "outer",on = ['Country Code', 'Year']) 
    else: 
        print("This did not have any feature to be selected: " + i)

This did not have any feature to be selected: error_2 Aid Effectiveness.csv
This did not have any feature to be selected: error_1 Economy & Growth.csv
This did not have any feature to be selected: error_2 Agriculture.csv
final Health.csv
Check


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


Check


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the 

Check


...

Check


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the 

Check


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


KeyboardInterrupt: 

In [17]:
dfcorrelated.to_csv('dfcorrelatedasasa.csv')

In [14]:
dfcorrelated.to_csv('dfcorrelated.csv')

In [88]:
dfAllAggregation.shape

(0, 2)

In [90]:
list(dfAllAggregationFeature30.columns)

['DT.MAT.OFFT',
 'DT.DOD.DSTC.XP.ZS',
 'DT.MAT.DPPG',
 'SP.POP.GROW',
 'SE.SEC.CUAT.LO.MA.ZS',
 'SL.TLF.0714.SW.TM',
 'DT.GPA.PRVT',
 'SE.ADT.1524.LT.FM.ZS',
 'SP.HOU.FEMA.ZS',
 'DT.DOD.DECT.EX.ZS',
 'SL.UEM.1524.FE.ZS',
 'EN.ATM.CO2E.EG.ZS',
 'SL.FAM.0714.ZS',
 'per_sa_allsa.cov_pop_tot',
 'DT.CUR.OTHC.ZS',
 'SG.DMK.ALLD.FN.ZS',
 'DT.INR.OFFT',
 'SL.SRV.0714.ZS',
 'per_allsp.adq_pop_tot',
 'FM.LBL.BMNY.ZG',
 'NE.CON.GOVT.KD.ZG',
 'SL.TLF.0714.WK.FE.TM',
 'ER.H2O.FWIN.ZS',
 'per_lm_alllm.cov_q1_tot',
 'Country Code',
 'Year',
 'GDP']

In [65]:
dfwona.to_csv('dfwona.csv')

In [15]:
dfCountry = pd.DataFrame()
dfIndicator = pd.DataFrame()
for i in os.listdir(): 
    if (('csv' in i) & ('final' in i)) :
        print(i)
        df = pd.read_csv( os.getcwd() + '/' + i)
        del df['Unnamed: 0']
        del df['V63']
        del df['V62']
        df.columns  = ['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code', '2003', '2004','2005', '2006', '2007', '2008', '2009','2010', '2011', '2012', '2013', '2014','2015', '2016']
        dfindicator = df[["Indicator Code", "Indicator Name"]].drop_duplicates()
        dfindicator["FileName"] = i
        dfcountry = df[["Country Code", "Country Name"]].drop_duplicates()  
        dfIndicator = dfIndicator.append(dfindicator)
        dfCountry = dfCountry.append(dfcountry)
dfIndicator = dfIndicator.drop_duplicates()
dfCountry = dfCountry.drop_duplicates()
                       

final Health.csv
final Poverty.csv
final Private Sector.csv
final Agriculture.csv
final Urban Development.csv
final Infrastructure.csv
final Trade.csv
final Public Sector.csv
final Financial Sector.csv
final Economy & Growth.csv
final Science & Technology.csv
final Aid Effectiveness.csv
final Environment.csv
final Energy & Mining.csv
final Social Development.csv
final Social Protection & Labour.csv
final Education.csv
final Gender.csv
final Climate Change.csv
final External Debt.csv


In [81]:
dfIndicator.to_csv('dfIndicator.csv')
dfCountry.to_csv('dfCountry.csv')


In [82]:
dfIndicator

,Indicator Code,Indicator Name
0,SP.UWT.TFRT,Unmet need for contraception (% of married wom...
264,SP.REG.DTHS.ZS,Completeness of death registration with cause-...
528,SP.REG.BRTH.ZS,Completeness of birth registration (%)
792,SP.REG.BRTH.UR.ZS,"Completeness of birth registration, urban (%)"
1056,SP.REG.BRTH.RU.ZS,"Completeness of birth registration, rural (%)"
1320,SP.REG.BRTH.MA.ZS,"Completeness of birth registration, male (%)"
1584,SP.REG.BRTH.FE.ZS,"Completeness of birth registration, female (%)"
1848,SP.POP.TOTL.MA.ZS,"Population, male (% of total)"
2376,SP.POP.TOTL.FE.ZS,"Population, female (% of total)"
3168,SP.POP.GROW,Population growth (annual %)
